<a href="https://colab.research.google.com/github/woongjoonchoi/DeepLearningPaper-Reproducing/blob/master/Vgg/vgg_model_define.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import albumentations as A
import numpy as np

import matplotlib.pyplot as plt
from torchvision.datasets import Caltech256 ,Caltech101 ,CIFAR100
import os
from PIL import Image
from urllib.request import urlretrieve
import requests
import tarfile

In [2]:
from torch import nn

In [3]:
import torch

## VGG model define

In [4]:
Config_channels = {
"A" : [64,"M" , 128,  "M"  , 256,256,"M" ,512,512 ,"M" , 512,512,"M"] ,
"A_lrn" : [64,"LRN","M" , 128,  "M"  , 256,256,"M" ,512,512 ,"M" , 512,512,"M"] ,
"B" :[64,64,"M" , 128,128,  "M"  , 256,256,"M" ,512,512 ,"M" , 512,512,"M"]  ,
"C" : [64,64,"M" , 128,128,  "M"  , 256,256,256,"M" ,512,512 ,512,"M" , 512,512,512,"M"] ,
"D" :[64,64,"M" , 128,128,  "M"  , 256,256,256,"M" ,512,512 ,512,"M" , 512,512,512,"M"] ,
"E" :[64,64,"M" , 128,128,  "M"  , 256,256,256,256,"M" ,512,512 ,512,512,"M" , 512,512,512,512,"M"]         ,

}

In [5]:
Config_kernel = {
"A" : [3,2 , 3,  2  , 3,3,2 ,3,3 ,2 , 3,3,2] ,
"A_lrn" : [3,2,2 , 3,  2  , 3,3,2 ,3,3 ,2 , 3,3,2] ,
"B" :[3,3,2 , 3,3,  2  , 3,3,2 ,3,3 ,2 , 3,3,2]  ,
"C" : [3,3,2 , 3,3,  2  , 3,3,1,2 ,3,3 ,1,2 , 3,3,1,2] ,
"D" :[3,3,2 , 3,3,  2  , 3,3,3,2 ,3,3 ,3,2 , 3,3,3,2] ,
"E" :[3,3,2 , 3,3,  2  , 3,3,3,3,2 ,3,3 ,3,3,2 , 3,3,3,3,2]         ,

}

In [6]:
def make_feature_extractor(cfg_c,cfg_k):
    feature_extract = []
    in_channels = 3
    i = 1
    for  out_channels , kernel in zip(cfg_c,cfg_k) :
        # print(f"{i} th layer {out_channels} processing")
        if out_channels == "M" :
            feature_extract += [nn.MaxPool2d(kernel,2) ]
        elif out_channels == "LRN":
            feature_extract += [nn.LocalResponseNorm(5,k=2) , nn.ReLU()]
        elif out_channels == 1:
            feature_extract+= [nn.Conv2d(in_channels,out_channels,kernel,stride = 1) , nn.ReLU()]
        else :
            feature_extract+= [nn.Conv2d(in_channels,out_channels,kernel,stride = 1 , padding = 1) , nn.ReLU()]

        if isinstance(out_channels,int) :   in_channels = out_channels
        i+=1
    return nn.Sequential(*feature_extract)


In [7]:
class Model_vgg(nn.Module) :
    def __init__(self,version , num_classes):
        conv_5_out_w ,conv_5_out_h = 7,7
        conv_5_out_dim =512
        conv_1_by_1_1_outchannel = 4096
        conv_1_by_1_2_outchannel = 4096
        # conv_1_by_1_3_outchannel = num_classes
        super().__init__()
        self.feature_extractor = make_feature_extractor(Config_channels[version] , Config_kernel[version])

        self.output_layer = nn.Sequential(
                             nn.Conv2d(conv_5_out_dim  ,conv_1_by_1_1_outchannel ,7) ,
                             nn.ReLU(),
                             nn.Dropout2d(),
                             nn.Conv2d(conv_1_by_1_1_outchannel ,conv_1_by_1_2_outchannel,1 ) ,
                             nn.ReLU(),
                             nn.Dropout2d(),
                             nn.Conv2d(conv_1_by_1_2_outchannel ,num_classes,1 )
                             )

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))

    def forward(self,x):
        x = self.feature_extractor(x)
        x = self.output_layer(x)
        x= self.avgpool(x)
        x= torch.flatten(x,start_dim = 1)
        return x


    #     pass



## Unit Test

### Model Parameter Unit Test

In [8]:
model_version = ["A","A_lrn" , "B","C","D","E"]

In [9]:
model_param_num  = [133,133,133,134,138,144]

In [10]:
import math


In [11]:
def parameter_num(model_parameters):
    summed = sum([torch.numel(p) for p in model_parameters])
    return summed

In [12]:
for v ,number in zip(model_version ,model_param_num):
    num_classes= 1000
    model_test = Model_vgg(v,num_classes)
    print(f"{v} model processing")
    print(round(parameter_num(model_test.parameters()) / 1e+6))
    assert(round(parameter_num(model_test.parameters()) / 1e+6) == number)

A model processing
133
A_lrn model processing
133
B model processing
133
C model processing
134
D model processing
138
E model processing
144


### Model outputshape test

In [13]:
model_version = ["A","A_lrn" , "B","C","D","E"]

In [14]:
random_shape =[(9,3,400,400)  ,  (1,3,400,400)  ,(1,3,224,224)  , (8,3,389,389)]

In [15]:
for sp in random_shape:
    for v in model_version:
        batch_size = sp[0]
        num_classes = 1000
        model_test = Model_vgg(v , num_classes)
        random_tensor = torch.rand(sp)
        output   = model_test(random_tensor)
        print(f'{v} model output shape :{output.shape}')
        assert tuple(output.shape) == (batch_size,num_classes)

A model output shape :torch.Size([9, 1000])
A_lrn model output shape :torch.Size([9, 1000])
B model output shape :torch.Size([9, 1000])
C model output shape :torch.Size([9, 1000])
D model output shape :torch.Size([9, 1000])
E model output shape :torch.Size([9, 1000])
A model output shape :torch.Size([1, 1000])
A_lrn model output shape :torch.Size([1, 1000])
B model output shape :torch.Size([1, 1000])
C model output shape :torch.Size([1, 1000])
D model output shape :torch.Size([1, 1000])
E model output shape :torch.Size([1, 1000])
A model output shape :torch.Size([1, 1000])
A_lrn model output shape :torch.Size([1, 1000])
B model output shape :torch.Size([1, 1000])
C model output shape :torch.Size([1, 1000])
D model output shape :torch.Size([1, 1000])
E model output shape :torch.Size([1, 1000])
A model output shape :torch.Size([8, 1000])
A_lrn model output shape :torch.Size([8, 1000])
B model output shape :torch.Size([8, 1000])
C model output shape :torch.Size([8, 1000])
D model output s

In [25]:
model_test = Model_vgg("A_lrn" , num_classes)

In [19]:
from torchsummary import summary

### weight initialize

In [46]:
@torch.no_grad()
def initial(m):
    if isinstance(m,nn.Conv2d):
        print(m)
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None :
            nn.init.zeros_(m.bias)

In [47]:
model_test.apply(initial)

Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(512, 4096, kernel_size=(7, 7), stride=(1, 1))
Conv2d(4096, 4096, kernel_size=(1, 1), stride=(1, 1))
Conv2d(4096, 1000, kernel_size=(1, 1), stride=(1, 1))


Model_vgg(
  (feature_extractor): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (13): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU()
    (15): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (16): ReLU()
    (17): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=

In [40]:
for name,module in model_test.named_parameters() :
    print(name)

feature_extractor.0.weight
feature_extractor.0.bias
feature_extractor.5.weight
feature_extractor.5.bias
feature_extractor.8.weight
feature_extractor.8.bias
feature_extractor.10.weight
feature_extractor.10.bias
feature_extractor.13.weight
feature_extractor.13.bias
feature_extractor.15.weight
feature_extractor.15.bias
feature_extractor.18.weight
feature_extractor.18.bias
feature_extractor.20.weight
feature_extractor.20.bias
output_layer.0.weight
output_layer.0.bias
output_layer.3.weight
output_layer.3.bias
output_layer.6.weight
output_layer.6.bias


### model summarize


In [41]:
#
# model_test.apply(torch.nn.init.xavier_uniform_)
summary(model_test,(3,224,224),batch_size=512)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [512, 64, 224, 224]           1,792
              ReLU-2        [512, 64, 224, 224]               0
 LocalResponseNorm-3        [512, 64, 224, 224]               0
              ReLU-4        [512, 64, 224, 224]               0
         MaxPool2d-5        [512, 64, 112, 112]               0
            Conv2d-6       [512, 128, 112, 112]          73,856
              ReLU-7       [512, 128, 112, 112]               0
         MaxPool2d-8         [512, 128, 56, 56]               0
            Conv2d-9         [512, 256, 56, 56]         295,168
             ReLU-10         [512, 256, 56, 56]               0
           Conv2d-11         [512, 256, 56, 56]         590,080
             ReLU-12         [512, 256, 56, 56]               0
        MaxPool2d-13         [512, 256, 28, 28]               0
           Conv2d-14         [512, 512,

In [ ]:
output = model_test(torch.rand(random_shape[1]))



In [ ]:
bb = nn.Softmax(output)

In [ ]:
dir(bb)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__constants__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_compiled_call_impl',
 '_forward_hooks',
 '_forward_hooks_always_called',
 '_forward_hooks_with_kwargs',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_per

In [ ]:
target = torch.rand(1,1000)

In [ ]:
loss = nn.CrossEntropyLoss()(bb,target)

TypeError: cross_entropy_loss(): argument 'input' (position 1) must be Tensor, not Softmax

In [ ]:
loss

tensor(3509.0122, grad_fn=<DivBackward1>)